In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('TlkPersonaChatRus\dialogues.tsv', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
from bs4 import BeautifulSoup as bs
html = bs(text,'lxml')

In [4]:
participant = html.find_all('span')

In [5]:
import re

phrases = list()
dialog = list()
last_num_person = 0
dialog.append('<next>')

for message in participant:
    num_person = re.findall(r'Пользователь ([1,2]):', message.text)
    if num_person:       
        phrases.append(re.findall(f'Пользователь {num_person[0]}: (.*)', message.text)[0])
        if num_person[0] == last_num_person:
            dialog[-1] = f"{dialog[-1]} {re.findall(f'Пользователь {num_person[0]}: (.*)', message.text)[0]}"
        else:
            dialog.append(re.findall(f'Пользователь {num_person[0]}: (.*)', message.text)[0])
            
        last_num_person = num_person[0]
    else:
        if dialog[-1] != '<next>':
            dialog.append('<next>')
        last_num_person = 0

In [6]:
question = list()
answer = list()

for i in range(len(dialog) - 1):
    if dialog[i] == '<next>' or dialog[i+1] == '<next>':
        continue
    question.append(dialog[i])
    answer.append(dialog[i+1])

In [7]:
question[:20]

['Привет) расскажи о себе',
 'Привет) под вкусный кофеек настроение поболтать появилось)',
 'Что читаешь? Мне нравится классика Я тоже люблю пообщаться',
 'Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю',
 'А я выращиваю фиалки И веду здоровый и активный образ жизни!',
 'Ух ты, интересно.',
 'Ты случайно не принц на белом коне? Я его очень жду..',
 'Привет!',
 'Привет,Как жизнь?',
 'Отлично) Солнышко светит, птички поют!',
 'Я вот сегодня понял, что меня тупо используют, всемнужны от меня лишь деньги, ненавижу людей Чем занимаешься по жизни, я вот бизнесмен.',
 'А я вот учу детей, работаю с начальными классами Не все люди такие, как ты говоришь Помимо работы чем еще ты занимаешься?',
 'К свадьбе готовлюсь А ты?',
 'Вот видишь) значит, нашел такую женщину, которой ненужны от тебя деньги',
 'Привет Как дела ?',
 'Добрый день! Хорошо,  чем увлекаетесь?',
 'Я бегаю по утрам а ты? Есть любимые вещи или еда ? Занят ?',
 'Я люблю петь в караоке)',
 'Круто )',
 'Люблю гот

In [8]:
answer[:20]

['Привет) под вкусный кофеек настроение поболтать появилось)',
 'Что читаешь? Мне нравится классика Я тоже люблю пообщаться',
 'Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю',
 'А я выращиваю фиалки И веду здоровый и активный образ жизни!',
 'Ух ты, интересно.',
 'Ты случайно не принц на белом коне? Я его очень жду..',
 'А у меня из хобби каждую неделю тусить с моим лучшимдругом)',
 'Привет,Как жизнь?',
 'Отлично) Солнышко светит, птички поют!',
 'Я вот сегодня понял, что меня тупо используют, всемнужны от меня лишь деньги, ненавижу людей Чем занимаешься по жизни, я вот бизнесмен.',
 'А я вот учу детей, работаю с начальными классами Не все люди такие, как ты говоришь Помимо работы чем еще ты занимаешься?',
 'К свадьбе готовлюсь А ты?',
 'Вот видишь) значит, нашел такую женщину, которой ненужны от тебя деньги',
 'Да я надеюсь на это,люблю ее',
 'Добрый день! Хорошо,  чем увлекаетесь?',
 'Я бегаю по утрам а ты? Есть любимые вещи или еда ? Занят ?',
 'Я люблю петь в

# Первая модель, которая превратит текст в человеческий

In [9]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [10]:
batch_size = 64
epochs = 60
latent_dim = 256
num_samples = 10000

In [11]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

target_texts = phrases[:4000].copy()

sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

input_texts = list(map(preprocess_text, target_texts))

for i in range(len(target_texts)):
    target_texts[i] = '\t' + target_texts[i] + '\n'

print(input_texts[:10])
print(target_texts[:10])

['привет рассказать', 'привет вкусный кофеёк настроение поболтать появиться', 'читать нравиться классика', 'любить пообщаться', 'любить животное обожать работа', 'фантастика любить', 'выращивать фиалка', 'вести здоровый активный образ жизнь', 'ух интересно', 'случайнон принц белый конь ждать']
['\tПривет) расскажи о себе\n', '\tПривет) под вкусный кофеек настроение поболтать появилось)\n', '\tЧто читаешь? Мне нравится классика\n', '\tЯ тоже люблю пообщаться\n', '\tЛюблю животных, просто обожаю, как и свою работу)\n', '\tЯ фантастику люблю\n', '\tА я выращиваю фиалки\n', '\tИ веду здоровый и активный образ жизни!\n', '\tУх ты, интересно.\n', '\tТы случайно не принц на белом коне? Я его очень жду..\n']


In [12]:
# Собираем из текстов токены и делаем pne-hot вектора на каждый токен

input_characters = set()
target_characters = set()
    
for i in range(len(target_texts)):
    for char in input_texts[i]:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_texts[i]:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [13]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [14]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [15]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    #print(target_text)
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [16]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

Epoch 1/60


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[functional_1/lstm/PartitionedCall]] [Op:__inference_train_function_4769]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


for seq_index in range(1):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)